In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
#from matplotlib import rcParam
import seaborn as sb
import os
import sys
sys.setrecursionlimit(1000000)
from scipy.stats import spearmanr
import pathlib
import scipy as sp
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache()
from scipy.stats import pearsonr
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import scanpy as sc
import anndata as ad

In [ ]:
from matplotlib import rcParams
#for_binary_barcode_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/filtered_barcode_matrix_for_binary_junk_test.pkl')
#normalised_barcode_path_junk_test = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/barcode_matrix_normalised_junk_test.pkl')
normalised_barcode_path= pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA32.6a/Sequencing/Source_target_together/source_filtered_thresholded.pkl')
#normalised_barcode_path= pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/Final_processed_sequences/barcode_matrix_normalised.pkl')
lcm_reg_dir = pathlib.PurePath('/nemo/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration')
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
# areas = areas.merge(RTtosample, how='inner', on='sample')
#areas.sort_values("RT_primer", inplace=True)





In [ ]:
#load barcodes, and drop samples that have been qc'd out
barcodes_across_sample = pd.read_pickle(normalised_barcode_path)

areas =areas[areas['sample'].isin(barcodes_across_sample.columns)]

In [ ]:
#drop columns that are zero in all values
#barcodes_matrix = barcodes_across_sample.loc[:, (barcodes_across_sample != 0).any(axis=0)]

In [ ]:
all_areas_fractions = pd.DataFrame(index=barcodes_across_sample.columns)
all_areas_fractions['name'] = 'string'
areas_to_look = areas.set_index('sample')
for i, row in areas_to_look.iterrows():
    row /= row.sum(0)
    
    if row[row.idxmax()]>0.9999:
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2))
    else:
        second_highest_row = row.drop(row.idxmax())
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2)) + ' ' + str(second_highest_row.idxmax()) + ' ' + str(row[second_highest_row.idxmax()].round(2))
    all_areas_fractions.loc[i] = name
    

In [ ]:
barcodes_across_sample_renamed = barcodes_across_sample.copy()
new_columns= {}
for val in barcodes_across_sample_renamed.columns:
    new_columns[val]= all_areas_fractions.loc[val]['name'] #create a dictionary to rename columns in the barcode matrix with tube names rather than RT sample names
barcodes_across_sample_renamed = barcodes_across_sample_renamed.rename(columns=new_columns)

In [ ]:
barcodes_across_sample_renamed =barcodes_across_sample_renamed.reset_index(drop=True)
sb.clustermap(barcodes_across_sample_renamed, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Greys", figsize=(15, 10), xticklabels=1 ,yticklabels=False, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
barcodes_no_source = barcodes_across_sample_renamed.drop(columns =['AUDv 0.81 AUDp 0.15','AUDp 0.71 AUDd 0.22', 'AUDp 1.0 AUDd 0.0', 'AUDd 0.62 SSs 0.34', 'TEa 0.41 AUDv 0.36'])

In [ ]:
barcodes_no_source = barcodes_no_source.loc[:, (barcodes_no_source != 0).any(axis=0)]

In [ ]:
barcodes_no_source = barcodes_no_source.loc[~(barcodes_no_source == 0).all(axis=1)]

In [ ]:

sb.clustermap(barcodes_no_source, metric='euclidean', standard_scale=0, norm=LogNorm(), cmap="Greys", figsize=(15, 10), xticklabels=1 ,yticklabels=False, cbar_pos=(1.01, 0.5, 0.02, 0.18))

In [ ]:
barcodes_across_sample_renamed.max(axis=1)

In [ ]:
barcodes_normalised_no_source = barcodes_across_sample_renamed.div(barcodes_across_sample_renamed.max(axis=1), axis=0)

In [ ]:
normalised_with_source = barcodes_across_sample_renamed.div(barcodes_across_sample_renamed.max(axis=1), axis=0)

In [ ]:
barcodes_across_sample_renamed

In [ ]:
#now drop source columns
barcodes_normalised_no_source = barcodes_normalised_no_source.loc[:, (barcodes_normalised_no_source != 1).all()]

#remove maximum values of source sites for clustering
#barcodes_normalised_no_source = barcodes_normalised.replace(1, 0)

In [ ]:
barcodes_normalised_no_source= barcodes_normalised_no_source.loc[:, (barcodes_normalised_no_source != 0).any(axis=0)]

In [ ]:
barcodes_normalised_no_source = barcodes_normalised_no_source.loc[~(barcodes_normalised_no_source == 0).all(axis=1)]

In [ ]:
barcodes_normalised_no_source

In [ ]:
# kmeans_labels = cluster.KMeans(n_clusters=6).fit_predict(barcodes_normalised_no_source)

In [ ]:
# Kmean = cluster.KMeans(n_clusters=3)
# Kmean.fit(barcodes_no_source)

In [ ]:
reducer = umap.UMAP()

In [ ]:
scaled_barcode_data = StandardScaler().fit_transform(barcodes_normalised_no_source)

In [ ]:
embedding = reducer.fit_transform(scaled_barcode_data)
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1]
    )
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the MAPseq dataset', fontsize=12);

In [ ]:
clusterable_embedding = umap.UMAP(
    n_neighbors=100,
    min_dist=0.7,
    n_components=2,
    
).fit_transform(barcodes_normalised_no_source)
plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], s=1
            );

In [ ]:
labels = hdbscan.HDBSCAN(
    min_samples=5,
    min_cluster_size=10,
).fit_predict(clusterable_embedding)

In [ ]:
np.unique(labels)

In [ ]:
clustered = (labels >= 0)
clustered = (labels >= 0)
plt.scatter(clusterable_embedding[~clustered, 0],
            clusterable_embedding[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=3,
            alpha=0.5)
plt.scatter(clusterable_embedding[clustered, 0],
            clusterable_embedding[clustered, 1],
            c=labels[clustered],
            s=3,
            cmap='tab10');

plt.title('UMAP clustering of A1 MAPseq data')
plt.colorbar(ticks = [0,1,2, 3, 4, 5, 6], values = [0,1,2, 3, 4, 5, 6], label='cluster')     

In [ ]:
clustered = (labels >= 0)
clustered = (labels >= 0)
plt.scatter(clusterable_embedding[~clustered, 0],
            clusterable_embedding[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=3,
            alpha=0.5)
plt.scatter(clusterable_embedding[clustered, 0],
            clusterable_embedding[clustered, 1],
            c=labels[clustered],
            s=3,
            cmap='tab10');

plt.title('UMAP clustering of A1 MAPseq data')
plt.colorbar(ticks = [0,1,2, 3, 4, 5], values = [0,1,2, 3, 4, 5], label='cluster')          

In [ ]:
#redo clustering with sample numbers rather than area names
barcodes_normalised = barcodes_across_sample.div(barcodes_across_sample.max(axis=1), axis=0)

In [ ]:
#now drop source columns
barcodes_normalised_no_source = barcodes_normalised.loc[:, (barcodes_normalised != 1).all()]

In [ ]:
clusterable_embedding = umap.UMAP(
    n_neighbors=100,
    min_dist=0.7,
    n_components=2,
    
).fit_transform(barcodes_normalised_no_source)
plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], s=1
            );

In [ ]:
labels = hdbscan.HDBSCAN(
    min_samples=5,
    min_cluster_size=10,
).fit_predict(clusterable_embedding)

In [ ]:
np.unique(labels)

In [ ]:
clustered = (labels >= 0)
clustered = (labels >= 0)
plt.scatter(clusterable_embedding[~clustered, 0],
            clusterable_embedding[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=3,
            alpha=0.5)
plt.scatter(clusterable_embedding[clustered, 0],
            clusterable_embedding[clustered, 1],
            c=labels[clustered],
            s=3,
            cmap='tab20');

plt.title('UMAP clustering of A1 MAPseq data')
plt.colorbar(ticks = [0,1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], values = [0,1,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], label='cluster')     

In [ ]:
barcodes_normalised_no_source['labels'] = labels
barcodes_normalised_no_source =barcodes_normalised_no_source.set_index('labels')


In [ ]:
barcodes_normalised


In [ ]:
barcodes_normalised_no_source.to_pickle(normalised_barcode_path.parents[0]/'barcodes_with_cluster_label.pkl')

In [ ]:
#also save a copy with source sites
barcodes_normalised['labels'] = labels
barcodes_normalised =barcodes_normalised.set_index('labels')
barcodes_normalised.to_pickle(normalised_barcode_path.parents[0]/'barcodes_with_cluster_label_with_source.pkl')

In [ ]:
barcodes_normalised_no_source =barcodes_normalised_no_source.set_index('labels')

Lets use scampy to perform clustering using leiden algorithm

In [ ]:
#drop the thalamus samples
barcodes_across_sample_no_thal = barcodes_across_sample.drop([44, 30, 54, 56, 66, 67], axis=1)

In [ ]:
barcodes_across_sample_no_thal

In [ ]:
normalised_with_source = barcodes_across_sample_no_thal.div(barcodes_across_sample_no_thal.max(axis=1), axis=0)

In [ ]:
#now drop source columns
barcodes_normalised_no_source = normalised_with_source.loc[:, (normalised_with_source != 1).all()]

In [ ]:
adata = sc.AnnData(barcodes_normalised_no_source, 
    barcodes_normalised_no_source.index.to_frame(), 
    barcodes_normalised_no_source.columns.to_frame())

In [ ]:
dataset =ad.AnnData(barcodes_normalised_no_source)

In [ ]:
sc.tl.pca(dataset)
sc.pp.neighbors(dataset)
sc.tl.umap(dataset, spread=0.8)
sc.tl.leiden(dataset)
sc.tl.paga(dataset)
sc.pl.paga_compare(dataset, edges=True, threshold=0.05)

In [ ]:
sc.pl.paga_compare(dataset, edges=True, threshold=0.05)

In [ ]:
sc.tl.leiden(dataset, key_added = "leiden_1.0") 

In [ ]:
sc.tl.leiden(dataset, resolution = 0.6, key_added = "leiden_0.6")
sc.tl.leiden(dataset, resolution = 0.4, key_added = "leiden_0.4")
sc.tl.leiden(dataset, resolution = 1.4, key_added = "leiden_1.4")

In [ ]:
sc.pl.umap(dataset, color=['leiden_0.4', 'leiden_0.6', 'leiden_1.0','leiden_1.4'])

In [ ]:
sc.pl.umap(dataset, color=['leiden_0.6', 'leiden_1.0'], palette='tab20b', s=70)

In [ ]:
sc.pl.umap(dataset, color=['leiden_1.0'], palette='tab20b', s=70, title='A1 Single Neuron Projection Patterns')


In [ ]:
leiden_categories = pd.DataFrame(dataset.obs['leiden_1.0'])

In [ ]:
leiden_categories['leiden_1.0']=='1'

In [ ]:
barcodes_across_sample_renamed_1 = barcodes_across_sample_renamed.copy()
barcodes_across_sample_renamed_1['leiden'] = leiden_categories['leiden_1.0']

In [ ]:
df = barcodes_across_sample_renamed[leiden_categories['leiden_1.0']=='1']
df = df.loc[:, (df != 0).any(axis=0)]
df.loc[:, (df != 0).any(axis=0)]

In [ ]:
barcodes_across_sample_renamed_1

In [ ]:
normalised_with_source['leiden'] = leiden_categories['leiden_1.0']

In [ ]:
normalised_with_source = normalised_with_source.set_index('leiden')

In [ ]:
normalised_with_source.to_pickle(normalised_barcode_path.parents[0]/'barcodes_with_leiden_cluster_label_with_source.pkl')

Lets try to cluster visual areas only

In [ ]:
all_areas_fractions_vis =all_areas_fractions[all_areas_fractions['name'].str.contains('VIS')]
all_areas_fractions_vis = all_areas_fractions_vis[~all_areas_fractions_vis['name'].str.contains('VISC|Contra-VISa|Contra-VISam|Contra-VISC')]

In [ ]:
normalised_with_source = barcodes_across_sample.div(barcodes_across_sample.max(axis=1), axis=0)

In [ ]:
#now drop source columns
barcodes_normalised_no_source = normalised_with_source.loc[:, (normalised_with_source != 1).all()]

In [ ]:
visual_areas_only = barcodes_normalised_no_source[all_areas_fractions_vis.index.values]

In [ ]:
dataset =ad.AnnData(visual_areas_only)

In [ ]:
sc.tl.pca(dataset)
sc.pp.neighbors(dataset)
sc.tl.umap(dataset)
sc.tl.leiden(dataset)
sc.tl.paga(dataset)
sc.pl.paga_compare(dataset, edges=True, threshold=0.05)

In [ ]:
leiden_categories = pd.DataFrame(dataset.obs['leiden'])

In [ ]:
barcodes_across_sample_renamed[leiden_categories['leiden']=='1']